In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import pandas as pd
import numpy as np


try:

    df_happiness = pd.read_csv("World-happiness-report-updated_2024.csv", encoding='latin-1')
    df_tourism = pd.read_csv("world_tourism_economy_data.csv", encoding='latin-1')
    df_flights = pd.read_csv("Scraped_dataset.csv", encoding='latin-1')

    print("Files loaded successfully with 'latin-1' encoding. Checking initial rows...")
    print("\nDF Happiness Head:\n", df_happiness.head(2))
    print("\nDF Tourism Head:\n", df_tourism.head(2))
    print("\nDF Flights Head:\n", df_flights.head(2))

except Exception as e:
    print(f"Error loading files. Please check the exact file names: {e}")


Files loaded successfully with 'latin-1' encoding. Checking initial rows...

DF Happiness Head:
   Country name  year  Life Ladder  Log GDP per capita  Social support  \
0  Afghanistan  2008        3.724               7.350           0.451   
1  Afghanistan  2009        4.402               7.509           0.552   

   Healthy life expectancy at birth  Freedom to make life choices  Generosity  \
0                              50.5                         0.718       0.164   
1                              50.8                         0.679       0.187   

   Perceptions of corruption  Positive affect  Negative affect  
0                      0.882            0.414            0.258  
1                      0.850            0.481            0.237  

DF Tourism Head:
                        country country_code  year  tourism_receipts  \
0                        Aruba          ABW  1999      7.820000e+08   
1  Africa Eastern and Southern          AFE  1999      8.034209e+09   

   tourism_

In [ ]:

import pandas as pd

try:
    df_tourism = pd.read_csv("world_tourism_economy_data.csv", encoding='latin-1')
    df_flights = pd.read_csv("Scraped_dataset.csv", encoding='latin-1')

    print("--- Tourism Economy Columns (df_tourism) ---")
    print(df_tourism.columns.tolist())

    print("\n--- Scraped Flight Columns (df_flights) ---")
    print(df_flights.columns.tolist())

except Exception as e:
    print(f"Error reading files for column inspection: {e}")

--- Tourism Economy Columns (df_tourism) ---
['country', 'country_code', 'year', 'tourism_receipts', 'tourism_arrivals', 'tourism_exports', 'tourism_departures', 'tourism_expenditures', 'gdp', 'inflation', 'unemployment']

--- Scraped Flight Columns (df_flights) ---
['Date of Booking', 'Date of Journey', 'Airline-Class', 'Departure Time', 'Arrival Time', 'Duration', 'Total Stops', 'Price']


In [ ]:
print("\n--- Cleaning Happiness Data  ---")

# 1. Standardize and select columns
df_happiness.rename(columns={
    'Country name': 'Country',
    'year': 'Year',
    'Life Ladder': 'Happiness_Score',
    'Log GDP per capita': 'Log_GDP',
    'Freedom to make life choices': 'Freedom_Score'
}, inplace=True)

# 2. Cleaning and filtering for the latest year
df_happiness.dropna(subset=['Happiness_Score', 'Country', 'Year'], inplace=True)
df_happiness['Country'] = df_happiness['Country'].str.strip()

# Getting the latest year available in happiness data (which covers 2005-2024)
LATEST_YEAR = df_happiness['Year'].max()

df_happiness_final = df_happiness[df_happiness['Year'] == LATEST_YEAR].copy()

# 3. Selecting final columns for merging
df_happiness_final = df_happiness_final[['Country', 'Happiness_Score', 'Log_GDP', 'Freedom_Score']].copy()
df_happiness_final.rename(columns={'Happiness_Score': f'Happiness_Score_{LATEST_YEAR}'}, inplace=True)

print(f"Happiness Data Cleaned. Final dataframe (df_happiness_final) created using data from Year: {LATEST_YEAR}")


--- Cleaning Happiness Data  ---
Happiness Data Cleaned. Final dataframe (df_happiness_final) created using data from Year: 2023


In [ ]:
print("\n--- Cleaning Tourism Data (Final Simplified) ---")

# Columns identified: 'country', 'year', 'tourism_arrivals', 'tourism_receipts'
df_tourism.dropna(subset=['country', 'year', 'tourism_arrivals', 'tourism_receipts'], inplace=True)

# Ensuring Year is numeric
df_tourism['year'] = pd.to_numeric(df_tourism['year'], errors='coerce', downcast='integer')
df_tourism['country'] = df_tourism['country'].str.strip()

# Since the latest year is 2020, filtering for only 2020 data
LATEST_TOURISM_YEAR = df_tourism['year'].max()
df_tourism_final = df_tourism[df_tourism['year'] == LATEST_TOURISM_YEAR].copy()

# Selecting only the columns needed for merging
df_tourism_final = df_tourism_final[['country', 'tourism_arrivals', 'tourism_receipts']].copy()

# Renaming columns for clarity in the final file
df_tourism_final.rename(columns={
    'country': 'Country',
    'tourism_arrivals': f'Visitors_{LATEST_TOURISM_YEAR}',
    'tourism_receipts': f'Tourism_Revenue_{LATEST_TOURISM_YEAR}_USD'
}, inplace=True)

print(f"Tourism Data Cleaned. Using metrics from the latest available year: {LATEST_TOURISM_YEAR}")


--- Cleaning Tourism Data (Final Simplified) ---
Tourism Data Cleaned. Using metrics from the latest available year: 2020


In [ ]:
print("\n--- Cleaning Flight Price Data  ---")

df_flights.rename(columns={'Price': 'Flight_Price_USD'}, inplace=True)

# 1. Cleaning the Price column (Remove commas and convert to numeric)
df_flights['Flight_Price_USD'] = (
    df_flights['Flight_Price_USD']
    .astype(str) # Convert to string first, just in case
    .str.replace(',', '', regex=False)
    .astype(float)
)

# 2. Converting Date of Journey (Using DD/MM/YYYY format based on your raw data)
df_flights['Date of Journey'] = pd.to_datetime(
    df_flights['Date of Journey'],
    format='%d/%m/%Y',
    errors='coerce' # Convert any bad dates to NaT
)

# 3. Dropping rows with missing core data (Price or Date)
df_flights.dropna(subset=['Flight_Price_USD', 'Date of Journey'], inplace=True)

valid_rows = df_flights.shape[0]
print(f"Total valid flight data rows after cleaning: {valid_rows}")


if valid_rows > 0:
    #  Feature Engineering: Seasonality
    df_flights['Travel_Month'] = df_flights['Date of Journey'].dt.month

    # Aggregating for Final Analysis File (Average Price by Month)
    df_flights_agg = df_flights.groupby(['Travel_Month']).agg(
        Avg_Flight_Price=('Flight_Price_USD', 'mean'),
        Total_Samples=('Flight_Price_USD', 'count')
    ).reset_index()

    # Rerun the export for this file
    df_flights_agg.to_csv("flights_seasonal_cleaned.csv", index=False)
    print(f"✅ Flight Price Data Successfully Cleaned and Saved: {df_flights_agg.shape[0]} rows.")

else:
    print("FATAL ERROR: Flight data is still unusable. Skipping seasonality analysis.")
    df_flights_agg = pd.DataFrame()


--- Cleaning Flight Price Data  ---
Total valid flight data rows after cleaning: 452088
✅ Flight Price Data Successfully Cleaned and Saved: 3 rows.


In [ ]:
print("\n--- Final Merging and Exporting  ---")


# 1. Creating the Country Master File (Merge Tourism and Happiness)
df_master_country = pd.merge(
    df_tourism_final,
    df_happiness_final,
    on='Country',
    how='inner'
)

# 2. Export the final files (Creating them in the Colab root)
df_master_country.to_csv("travel_master_country_data.csv", index=False)
print(f"✅ Master Country Data saved: travel_master_country_data.csv ({df_master_country.shape[0]} rows)")

df_flights_agg.to_csv("flights_seasonal_cleaned.csv", index=False)
print(f"✅ Flights/Seasonal Data saved: flights_seasonal_cleaned.csv ({df_flights_agg.shape[0]} rows)")

print("\nCleaning Phase Complete! Download these two files for Power BI.")


--- Final Merging and Exporting  ---
✅ Master Country Data saved: travel_master_country_data.csv (61 rows)
✅ Flights/Seasonal Data saved: flights_seasonal_cleaned.csv (3 rows)

Cleaning Phase Complete! Download these two files for Power BI.
